In [4]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
from google.colab import drive, files # google colab specific
import requests
import pandas as pd
import os
import warnings
import sys
import matplotlib.pyplot as plt
import json
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf
from tensorflow import keras
from keras import layers
import math
from sklearn.metrics import mean_squared_error
from datetime import datetime
import datetime as dt
from pylab import rcParams
from numpy import array


In [9]:
# Read Data from CSV into Dataframe
df = pd.read_csv('/content/drive/MyDrive/JPM/jphomevalue/data/estated_output_MDF_tampa1.csv')
df.head()

,DATE,TPXRSA,"3511 E 29th Ave, Tampa, FL 33605","214 W Emily St, Tampa, FL 33603","2904 W Averill Ave, Tampa, FL 33611","2911 W Estrella St, Tampa, FL 33629","4118 W Olive St, Tampa, FL 33616","8716 N Greenwood Ave, Tampa, FL 33604","2412 E 20th Ave, Tampa, FL 33605","5828 Mariner St, Tampa, FL 33609",...,"2601 W Hillsborough Ave, Tampa, FL 33614","1022 S Frankland Rd, Tampa, FL 33629","105 E 26th Ave, Tampa, FL 33603","5120 W Poe Ave, Tampa, FL 33629","Tampa Bay Blvd, Tampa, FL 33614","4209 W Cayuga St, Tampa, FL 33614","6407 S Clark Ave, Tampa, FL 33616","15215 Amberly Dr APT 1008, Tampa, FL 33647","821 E McEwen Ave, Tampa, FL 33612","8512 N 11th St, Tampa, FL 33604"
0,1/1/2002,120.395793,58833.41085,64532.14505,162373.7928,260115.1108,103861.4374,65495.31139,28293.01136,742360.4596,...,357695.9010,1647616.427,75066.77694,1676511.418,320493.6010,328560.1191,154588.1982,58271.56381,46713.56768,68814.79540
1,1/2/2002,120.405607,58838.20672,64537.40546,162387.0289,260136.3144,103869.9038,65500.65032,28295.31769,742420.9741,...,357725.0590,1647750.735,75072.89610,1676648.080,320519.7264,328586.9021,154600.7997,58276.31389,46717.37560,68820.40492
2,1/3/2002,120.415421,58843.00260,64542.66588,162400.2650,260157.5180,103878.3702,65505.98925,28297.62403,742481.4885,...,357754.2170,1647885.042,75079.01525,1676784.743,320545.8518,328613.6851,154613.4011,58281.06397,46721.18351,68826.01444
3,1/4/2002,120.425236,58847.79848,64547.92630,162413.5011,260178.7216,103886.8366,65511.32818,28299.93037,742542.0029,...,357783.3751,1648019.350,75085.13441,1676921.406,320571.9772,328640.4680,154626.0025,58285.81404,46724.99142,68831.62396
4,1/5/2002,120.435050,58852.59435,64553.18671,162426.7372,260199.9252,103895.3030,65516.66711,28302.23671,742602.5173,...,357812.5331,1648153.657,75091.25357,1677058.069,320598.1027,328667.2510,154638.6040,58290.56412,46728.79934,68837.23349


In [53]:
# ********* PRIOR INITIALIZATIONS **************************
# *************** RUN ONCE *********************************

# Load the model to be used
#model = keras.models.load_model('/content/drive/MyDrive/JPM/jphomevalue/trained_models/lstm-model-tampa-24-15.h5')

# Create a skeleton dataframe that will be initialized with Daily Shiller Data
shiller_filePath = "/content/drive/MyDrive/JPM/jphomevalue/data/Model_Prediction_Tampa_12months.csv"
shiller_data = pd.read_csv(shiller_filePath, header = 0)

# Convert date and set index to datetime
#shiller_data['DATE'] = pd.to_datetime(shiller_data['DATE'])
#shiller_data = shiller_data.set_index('DATE')

# Set period and interpolate
#shiller_data.index = shiller_data.index.to_period("1D")
#shiller_data = shiller_data.resample('1D').interpolate()
m_df = shiller_data.copy()
m_df['DATE'] = pd.to_datetime(m_df['DATE'])
m_df = m_df.set_index('DATE')
m_df = m_df.resample('1D').interpolate()
m_df = m_df.resample('1M').interpolate() 

# After the new dataframe is completely filled with prediction data transform to CSV  
m_df.to_csv('Shiller12monthpredicted.csv')

# Download final CSV file and it is ready to populate to DB
files.download('Shiller12monthpredicted.csv')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# ********* AUTOMATION BEGINS HERE! ************************

# Each column represents a house and it's historical prices
# Isolate a Date|House pair and store it into another df

for i in range(df.shape[1]-1): 
   # Store Date|House Pair and store to selective_df
   #selective_df = df.iloc[:, 2+i]

   # Store the current column name ('current house address)
   current_house = df.columns.values[2+i]

   # Call auto_prediction() function to automatically predict new prices and append to a new dataframe
   #name = df.columns.values[3]
   df5 = df[['DATE', current_house]]

   df5['DATE'] = pd.to_datetime(df5['DATE'])
   df5 = df5.set_index('DATE')
   #df5.index = df5.index.to_period("1M")
   df5 = df5.resample('1M').interpolate()
   
   #df_temp = selective_df.tolist()
   #df_temp.extend(predictions)

   # Start appending to new main dataframe
   m_df[current_house] = df5[current_house]


# After the new dataframe is completely filled with prediction data transform to CSV  
m_df.to_csv('completeHousePriceDataTampa')

# Download final CSV file and it is ready to populate to DB
files.download('completeHousePriceDataTampa')

  


In [52]:
# After the new dataframe is completely filled with prediction data transform to CSV  
m_df.to_csv('completeHousePriceDataTampa.csv')

# Download final CSV file and it is ready to populate to DB
files.download('completeHousePriceDataTampa.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>